# Final Project

W261 Machine Learning at Scale<br>
12 December 2018

Wei Wang;
Alice Lam;
John Tabbone;
Noah Randolph

### Question Formulation

Alice Lam: "enhancing CTR means improved monetization of your current traffic (eyeballs/views). The algorithm to predict CTR accurately is useful for the platform to show specific ads to specific people who would have the highest CTR"

### Algorithm Explanation

### EDA & Discussion of Challenges

In [2]:
%%writefile eda.py
#!/usr/bin/env python

# start Spark Session
from pyspark.sql import SparkSession
app_name = "eda"
spark = SparkSession\
        .builder\
        .appName(app_name)\
        .getOrCreate()
        
sc = spark.sparkContext

# load the data into Spark dataframes
df = spark.read.csv('gs://w261_final_project/train.txt', sep='\t')

# split into test and training data
splits = df.randomSplit([0.2, 0.8], seed=2615)
testDf = splits[0]
trainDf = splits[1]

print(df.columns)
print(testDf.count())
print(trainDf.count())

Writing eda.py


In [3]:
!python submit_job_to_cluster.py --project_id=w261-222623 --zone=us-central1-b --cluster_name=testcluster --gcs_bucket=w261_final_project --key_file=$HOME/w261.json --create_new_cluster --pyspark_file=eda.py --instance_type=n1-standard-4 --worker_nodes=2

Traceback (most recent call last):
  File "/anaconda3/lib/python3.5/site-packages/googleapiclient/discovery_cache/__init__.py", line 36, in autodetect
    from google.appengine.api import memcache
ImportError: No module named 'google.appengine'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/anaconda3/lib/python3.5/site-packages/googleapiclient/discovery_cache/file_cache.py", line 33, in <module>
    from oauth2client.contrib.locked_file import LockedFile
ImportError: No module named 'oauth2client.contrib.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/anaconda3/lib/python3.5/site-packages/googleapiclient/discovery_cache/file_cache.py", line 37, in <module>
    from oauth2client.locked_file import LockedFile
ImportError: No module named 'oauth2client.locked_file'

During handling of the above exception, another exception occurred:

Traceback (

#### Results from running EDA code above:
number of testing data rows, $n_{test}=9164811$<br>
number of training data rows, $n=36675806$<br>

### Algorithm Implementation

### Application of Course Concepts